In [22]:
import requests

res = requests.post('https://api-seller.ozon.ru/v2/category/tree', headers={
    'Client-Id': '589954', 'Api-Key': '2e0076d1-1ecd-4105-b1df-d87e1a244648'
})

arr = res.json()['result']
categories = []

categories_by_id = {}

def parse(arr):
    for obj in arr:
        tmp = {}
        tmp['category_id'] = obj['category_id']
        tmp['title'] = obj['title']
        if obj['children'] and len(obj['children']) != 0:
            categories_by_id[tmp['category_id']] = obj['title']
            categories.append(tmp)
            parse(obj['children'])
        else:
            categories_by_id[tmp['category_id']] = obj['title']
            categories.append(tmp)


parse(arr)

print(len(categories))
# print(categories_by_id)


10406


In [1]:
%pip install beautifulsoup4
%pip install selenium
%pip install pyppeteer

In [38]:
from bs4 import BeautifulSoup as bs
import requests
import re
import asyncio
from bs4 import BeautifulSoup
from pyppeteer import launch
import os
from collections import deque


browser = None
async def get_page():
    browser = await launch()
    page = await browser.newPage()
    await page.setUserAgent('Mozilla/5.0 (Windows NT 5.1; rv:5.0) Gecko/20100101 Firefox/5.0')
    return page

page = await get_page()
base = 'https://www.ozon.ru'
categories_url = 'https://www.ozon.ru/category/'
used_links = {categories_url}
found_categories = set()


In [39]:
await page.goto(categories_url, {'waitUntil': 'networkidle0'})
page_content = await page.content()

def grab_categories(page_content):
    soup = BeautifulSoup(page_content)
    links = set()
    for a in soup.find_all('a', href=True):
        if re.fullmatch('\/category\/[^\/]+\\d+\/', a['href']) is not None:
            link = base + a['href']
            if link in found_categories:
                continue

            links.add(link)
            found_categories.add(link)

    return links


queue_categories = deque()
queue_categories.extend(list(grab_categories(page_content)))
found_categories = set()
used_categories = set()

In [43]:
used_categories = set(map(lambda s: s.strip(), filter(None, open(
    './used_categories.txt', 'r').readlines())))
found_categories = set(map(lambda s: s.strip(), filter(None, open(
    './found_categories.txt', 'r').readlines())))
queue_categories.clear()
queue_categories.extend(map(lambda s: s.strip(), filter(None, open(
    './queue_categories.txt', 'r').readlines())))
 
print(len(queue_categories))

7587


In [46]:
count = 0

while len(queue_categories) != 0:
    url = queue_categories.pop()
    if url in used_categories:
        continue
    used_categories.add(url)
    try:
        await page.goto(url, {'waitUntil': 'networkidle0'})
        page_content = await page.content()
        links = grab_categories(page_content)
        queue_categories.extend(list(links))
        print('done')
        
        count += 1
        if count == 3:
          open('./used_categories.txt', 'w+').write('\n'.join(used_categories), )
          open('./found_categories.txt', 'w+').write('\n'.join(found_categories))
          open('./queue_categories.txt', 'w+').write('\n'.join(queue_categories))
          print('write')
    except:
        print('timeout error!')
        queue_categories.extend(url)
        await browser.close()
        page = await get_page()
        




done
done
done
done


In [ ]:
await browser.close()

AttributeError: 'NoneType' object has no attribute 'close'